# Realiza marcações

In [1]:
def marca_area(img, coordenadaInicial, coordenadaFinal, cor, opacidade, label):
    
    cv2.putText(img, label, 
                (coordenadaInicial[0], coordenadaInicial[1] - 5), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                0.5, 
                cor, 
                2, 
                1)
    
    cv2.rectangle(img, coordenadaInicial, coordenadaFinal, cor, int(opacidade))

In [15]:
def area_pedunculo(event, x, y, flags, param):
    
    global xt, yt, xb, yb, botao, img, controle, coordenadas
    
    cor = 0
    
    if(controle == 1):
        
        cor = COR_MANGA
        label = "manga"
        
    elif(controle == 2):
        
        cor = COR_PEDUNCULO
        label = "pedunculo"
        
    else:
        
        cor = COR_PONTO
        label = "ponto"

    if event == cv2.EVENT_LBUTTONDOWN:
        
        botao = True
        xt, yt = x, y
        controle += 1
        
    elif event == cv2.EVENT_LBUTTONUP:
        
        botao = False
        xb, yb = x, y
        
        xt, xb = min(xt, xb), max(xt, xb)
        yt, yb = min(yt, yb), max(yt, yb)
            
        marca_area(imagem, (xt, yt), (xb, yb), cor, 2, label)
        coordenadas.append(((xt, yt), (xb, yb)))

    if botao:
        
        image_temp = imagem.copy()
        marca_area(image_temp, (xt, yt), (x, y), cor, 2, label)
        cv2.imshow('Imagem '+ str(idx), image_temp)
        
    else:
        
        cv2.imshow('Imagem '+ str(idx), imagem)

In [16]:
def visualiza_marcacoes(CAMINHO_IMAGEM, ARQUIVO_JSON, LARGURA, ALTURA):
    
    imagem = cv2.imread(CAMINHO_IMAGEM)
    
    if imagem is None:
        
        print("Imagem não encontrada!")
        
        return
    
    imagem = cv2.resize(imagem, (LARGURA, ALTURA))
    
    with open(ARQUIVO_JSON) as file:
    
        coordenadas = json.load(file)
    
    xt, yt = coordenadas['MangoCoordinates']['xt'], coordenadas['MangoCoordinates']['yt']
    xb, yb = coordenadas['MangoCoordinates']['xb'], coordenadas['MangoCoordinates']['yb']
    
    cv2.rectangle(imagem, (xt, yt), (xb, yb), (0, 0, 255), 1)
    
    xt, yt = coordenadas['PeduncleCoordinates']['xt'], coordenadas['PeduncleCoordinates']['yt']
    xb, yb = coordenadas['PeduncleCoordinates']['xb'], coordenadas['PeduncleCoordinates']['yb']
    
    cv2.rectangle(imagem, (xt, yt), (xb, yb), (0, 255, 0), 1)
    
    xt, yt = coordenadas['PointCoordinates']['xt'], coordenadas['PointCoordinates']['yt']
    xb, yb = coordenadas['PointCoordinates']['xb'], coordenadas['PointCoordinates']['yb']
    
    cv2.rectangle(imagem, (xt, yt), (xb, yb), (255, 0, 0), 1)
        
    cv2.imshow("Visualizacao", imagem)    

In [17]:
import cv2
import json
import os

botao = False
xt, yt = -1, -1  
xb, yb = -1, -1
coordenadas = []

COR_MANGA =     (0, 0, 255)
COR_PEDUNCULO = (0, 255, 0)
COR_PONTO =     (255, 0, 0)

LARGURA = 480
ALTURA  = 640


REDUCTION = 3

QUANTIDADE_IMAGEM = 1
idx = 1

PASTA = "VALIDA_LIDAR/"
SUBPASTA = "(1)/"

while(idx <= QUANTIDADE_IMAGEM):
    
    cv2.namedWindow('Imagem ' + str(idx))
    cv2.setMouseCallback('Imagem ' + str(idx), area_pedunculo)
    
    controle = 0
    
    caminho_imagem = PASTA + SUBPASTA + str(idx) + ".jpg"
    print(f"Caminho imagem -> {caminho_imagem}")
    
    imagem = cv2.imread(caminho_imagem)
    
    if imagem is None:
        
        print(f"Erro ao carregar a imagem {caminho_imagem}")
        
        idx += 1
        
        continue
        
    arquivo_json   = PASTA + SUBPASTA + str(idx) + ".json"
    
    if not os.path.exists(arquivo_json):
        
        print(f"Criando json -> {arquivo_json}")
            
        open(arquivo_json, 'w')
        
    print(f"Caminho json   -> {arquivo_json}")
    
    imagem = cv2.resize(imagem, (LARGURA, ALTURA))
    
    cv2.imshow('Imagem ' + str(idx), imagem)

    key = cv2.waitKey(0)

    if key == 27:
        
        break
        
    if key == ord('p'):
        
        with open(arquivo_json, 'r+') as file:
            
            file_content = file.read()
            
            if not file_content:
                
                data = {}
                
            else:
                
                file.seek(0)
                data = json.load(file)
            
            label_json = ""
            
            data['ReductionImage'] = {
                
                'value': REDUCTION
            }
            
            file.seek(0)
            json.dump(data, file, indent=4)
        
            for c in range(len(coordenadas)):
                
                if(c == 0):
                    
                    label_json = "MangoCoordinates"
                    
                elif(c == 1):
                    
                    label_json = "PeduncleCoordinates"
                    
                else:
                    
                    label_json = "PointCoordinates"
                    

                data[label_json] = {

                        'xt': coordenadas[c][0][0],
                        'yt': coordenadas[c][0][1],
                        'xb': coordenadas[c][1][0],
                        'yb': coordenadas[c][1][1]
                    }

            file.seek(0)
            json.dump(data, file, indent=4)

            xt, yt = -1, -1  
            xb, yb = -1, -1
            
            print(f"Imagem -> {caminho_imagem} salva")

            idx += 1
        
    if key == ord('l'):
        
        xt, yt = -1, -1  
        xb, yb = -1, -1
        coordenadas = [ ]
        
        print(f"Imagem: {caminho_imagem} limpa")
        imagem = cv2.imread(caminho_imagem)
        
    if key == ord('v'):
        
         while(True):
                    
            key2 = cv2.waitKey(1)

            if key2 == 27:

                cv2.destroyWindow('Visualizacao')
                break
        
            visualiza_marcacoes(caminho_imagem, arquivo_json, LARGURA, ALTURA)

cv2.destroyAllWindows()


Caminho imagem -> VALIDA_LIDAR/(1)/1.jpg
Caminho json   -> VALIDA_LIDAR/(1)/1.json
(103, 314) (332, 476)
(104, 311) (334, 476)
(106, 313) (336, 474)
(106, 312) (336, 475)
(107, 316) (335, 474)


# Visualiza marcações

In [16]:
import json
import cv2

qtd = 1
i = 1

while(i <= qtd):
    
    imagem = cv2.imread("VALIDA_LIDAR/(1)/" + str(i) + ".jpg")
    
    if imagem is None:
        
        i += 1
        
        continue
        
    print(i)
    
    imagem = cv2.resize(imagem, (500, 500))
    
    with open('VALIDA_LIDAR/(1)/' + str(i) + '.json') as file:
    
        coordenadas = json.load(file)
    
    xt, yt = coordenadas['MangoCoordinates']['xt'], coordenadas['MangoCoordinates']['yt']
    xb, yb = coordenadas['MangoCoordinates']['xb'], coordenadas['MangoCoordinates']['yb']
    
    cv2.rectangle(imagem, (xt, yt), (xb, yb), (0, 0, 255), 1)
    
    xt, yt = coordenadas['PeduncleCoordinates']['xt'], coordenadas['PeduncleCoordinates']['yt']
    xb, yb = coordenadas['PeduncleCoordinates']['xb'], coordenadas['PeduncleCoordinates']['yb']
    
    cv2.rectangle(imagem, (xt, yt), (xb, yb), (0, 255, 0), 1)
    
    xt, yt = coordenadas['PointCoordinates']['xt'], coordenadas['PointCoordinates']['yt']
    xb, yb = coordenadas['PointCoordinates']['xb'], coordenadas['PointCoordinates']['yb']
    
    cv2.rectangle(imagem, (xt, yt), (xb, yb), (255, 0, 0), 1)
        
    cv2.imshow("Marcada " + str(i), imagem)
    
    key = cv2.waitKey(0)

    if key == 27:
        
        break
        
    if key == ord('.'):
        
        cv2.destroyAllWindows()
        i += 1
        
    if key == ord(','):
        
        cv2.destroyAllWindows()
        i -= 1

cv2.destroyAllWindows()

1


In [13]:
1440/3


480.0

In [12]:
1920/3

640.0